In [1]:
import shap
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from alibi.explainers import AnchorTabular
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  train_test_split
from sklearn.feature_selection import SelectFromModel

/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/wind_Turbine/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-03 17:39:34.105325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
failures = pd.read_csv('../data/model_data/failures.csv',sep=',')
components = failures['Component'].unique()

In [5]:
encoder = LabelEncoder()
for component in components:
    globals()[f"{component}_df"] = pd.read_csv(f'../data/model_data/labelled_data_{component}.csv',sep=',')
    globals()[f"{component}_df"]['Turbine_ID'] = encoder.fit_transform(['Turbine_ID']*globals()[f"{component}_df"].shape[0])
    # set the date as the index
    globals()[f"{component}_df"] = globals()[f"{component}_df"].set_index('Timestamp')

In [6]:
class_target_name = "Failure (Target)"
for component in components:
    X = globals()[f"{component}_df"].drop(columns=['Component',class_target_name])
    y = globals()[f"{component}_df"][class_target_name]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    globals()[f"{component}_X_train"] = X_train
    globals()[f"{component}_X_test"] = X_test
    globals()[f"{component}_y_train"] = y_train
    globals()[f"{component}_y_test"] = y_test

In [7]:
model_name = "xgb"

In [8]:
for component in components:
    with open ('../main_pipeline/models_MLFLOW/selected-{}.pickle'.format(component), 'rb') as file:
        globals()[f"{model_name}_{component}"] = pickle.load(file)


In [9]:
for component in components:
    globals()[f"{model_name}_{component}"].fit(globals()[f"{component}_X_train"], globals()[f"{component}_y_train"])
    globals()[f"{component}_params"] = globals()[f"{model_name}_{component}"].get_params()
    globals()[f"{component}_selector"] = SelectFromModel(globals()[f"{model_name}_{component}"], threshold=-np.inf, prefit=True, max_features=globals()[f"{component}_params"]['max_features'])
    globals()[f"{component}_selected_features"] = globals()[f"{component}_X_train"].columns[globals()[f"{component}_selector"].get_support()]
    globals()[f"{component}_selected_features_train"] = globals()[f"{component}_X_train"][globals()[f"{component}_selected_features"]]
    globals()[f"{component}_selected_features_test"] = globals()[f"{component}_X_test"][globals()[f"{component}_selected_features"]]
    globals()[f"{component}_selected_features_train"].reset_index(drop=True, inplace=True)
    globals()[f"{component}_selected_features_test"].reset_index(drop=True, inplace=True)
    

In [10]:
for component in components:
    globals()[f"{model_name}_{component}"].fit(globals()[f"{component}_selected_features_train"], globals()[f"{component}_y_train"])

In [19]:
for component in components:
    globals()[f"{component}_predit_fn"] = lambda x: globals()[f"{model_name}_{component}"].predict(globals()[f"{component}_selector"].transform(x))
    globals()[f"{component}_explainer"] = AnchorTabular(globals()[f"{component}_predit_fn"], globals()[f"{component}_selected_features_train"].columns)
    globals()[f"{component}_explainer"].fit(globals()[f"{component}_selected_features_train"].to_numpy())


In [43]:
GEARBOX_predit_fn = lambda x: xgb_GEARBOX.predict(x)
GEARBOX_explainer = AnchorTabular(GEARBOX_predit_fn, GEARBOX_selected_features_train.columns)
GEARBOX_explainer.fit(GEARBOX_selected_features_train.to_numpy())

GENERATOR_predit_fn = lambda x: xgb_GENERATOR.predict(x)
GENERATOR_explainer = AnchorTabular(GENERATOR_predit_fn, GENERATOR_selected_features_train.columns)
GENERATOR_explainer.fit(GENERATOR_selected_features_train.to_numpy())

GENERATOR_BEARING_predit_fn = lambda x: xgb_GENERATOR_BEARING.predict(x)
GENERATOR_BEARING_explainer = AnchorTabular(GENERATOR_BEARING_predit_fn, GENERATOR_BEARING_selected_features_train.columns)
GENERATOR_BEARING_explainer.fit(GENERATOR_BEARING_selected_features_train.to_numpy())

HYDRAULIC_predit_fn = lambda x: xgb_HYDRAULIC_GROUP.predict(x)
HYDRAULIC_explainer = AnchorTabular(HYDRAULIC_predit_fn, HYDRAULIC_GROUP_selected_features_train.columns)
HYDRAULIC_explainer.fit(HYDRAULIC_GROUP_selected_features_train.to_numpy())

TRANSFORMER_predit_fn = lambda x: xgb_TRANSFORMER.predict(x)
TRANSFORMER_explainer = AnchorTabular(TRANSFORMER_predit_fn, TRANSFORMER_selected_features_train.columns)
TRANSFORMER_explainer.fit(TRANSFORMER_selected_features_train.to_numpy())

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': None, 'disc_perc': (25, 50, 75)},
  'version': '0.9.7.dev0'}
)

In [44]:
def time_convert(sec):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  return "Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),sec)

In [45]:
for component in components:
    globals()[f"{component}_test_np"] = globals()[f"{component}_selected_features_test"].to_numpy()
    globals()[f"{component}_test_length"] = time.time()
    globals()[f"anchors_{component}"] =[ ]
    


In [47]:
start_time = time.time()
for i, dp in enumerate(GEARBOX_test_np):
    anchors_GEARBOX.append(GEARBOX_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, test_length, time_convert(time_lapsed)), end="\r", flush=True)


In [ ]:
for index, elem in enumerate(anchors_GEARBOX):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

In [50]:
start_time = time.time()
for i, dp in enumerate(GENERATOR_test_np):
    anchors_GENERATOR.append(GENERATOR_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, test_length, time_convert(time_lapsed)), end="\r", flush=True)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


...

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [51]:
for index, elem in enumerate(anchors_GENERATOR):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: Grd_Prod_VoltPhse1_Avg > 57417.10 AND Grd_Prod_Freq_Avg <= 7199.80
Precision: 0.98
Coverage: 0.20
1
Anchor: Grd_Prod_VoltPhse1_Avg > 57562.10
Precision: 0.99
Coverage: 0.25
2
Anchor: Grd_Prod_VoltPhse1_Avg > 57417.10
Precision: 0.96
Coverage: 0.75
3
Anchor: Grd_Prod_VoltPhse1_Avg > 57417.10
Precision: 0.96
Coverage: 0.75
4
Anchor: Grd_Prod_VoltPhse1_Avg > 57483.00
Precision: 0.97
Coverage: 0.50
5
Anchor: Grd_Prod_VoltPhse1_Avg > 57417.10 AND Grd_Prod_Freq_Avg <= 7199.80
Precision: 0.98
Coverage: 0.20
6
Anchor: Grd_Prod_VoltPhse1_Avg > 57562.10
Precision: 0.96
Coverage: 0.25
7
Anchor: Grd_Prod_VoltPhse1_Avg > 57417.10 AND Nac_Temp_Avg <= 4298.00
Precision: 0.99
Coverage: 0.34
8
Anchor: Grd_Prod_VoltPhse1_Avg > 57483.00
Precision: 0.96
Coverage: 0.50
9
Anchor: Hyd_Oil_Temp_Avg <= 5095.00
Precision: 0.96
Coverage: 0.50
10
Anchor: Hyd_Oil_Temp_Avg <= 5863.00 AND Grd_Prod_Freq_Avg <= 7199.80
Precision: 0.96
Coverage: 0.26
11
Anchor: Grd_Prod_VoltPhse1_Avg > 57562.10
Precision: 0.9

In [52]:
start_time = time.time()
for i, dp in enumerate(GENERATOR_BEARING_test_np):
    anchors_GENERATOR_BEARING.append(GENERATOR_BEARING_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, test_length, time_convert(time_lapsed)), end="\r", flush=True)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [53]:
for index, elem in enumerate(anchors_GENERATOR_BEARING):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: Grd_Prod_VoltPhse2_Avg <= 57145.00
Precision: 1.00
Coverage: 0.25
1
Anchor: Pressure_Avg_Freq > 409.01
Precision: 0.97
Coverage: 0.50
2
Anchor: Grd_Prod_VoltPhse2_Avg <= 57227.10
Precision: 0.99
Coverage: 0.50
3
Anchor: HVTrafo_Phase1_Temp_Avg <= 10354.00
Precision: 1.00
Coverage: 0.75
4
Anchor: HVTrafo_Phase3_Temp_Avg <= 9698.00
Precision: 1.00
Coverage: 0.51
5
Anchor: Cont_VCP_WtrTemp_Avg <= 5717.00
Precision: 0.99
Coverage: 0.76
6
Anchor: Amb_Temp_Avg > 2980.00
Precision: 0.99
Coverage: 0.50
7
Anchor: Nac_Direction_Avg <= 23844.40
Precision: 0.99
Coverage: 0.51
8
Anchor: HVTrafo_Phase1_Temp_Avg <= 10354.00
Precision: 0.99
Coverage: 0.75
9
Anchor: Grd_Prod_VoltPhse3_Avg > 57185.30
Precision: 0.99
Coverage: 0.50
10
Anchor: Grd_Prod_VoltPhse3_Avg > 57366.30
Precision: 1.00
Coverage: 0.25
11
Anchor: HVTrafo_Phase3_Temp_Avg <= 9698.00
Precision: 1.00
Coverage: 0.50
12
Anchor: Hyd_Oil_Temp_Avg <= 5863.00
Precision: 0.99
Coverage: 0.75
13
Anchor: HVTrafo_Phase1_Temp_Avg <= 9234.0

In [54]:
start_time = time.time()
for i, dp in enumerate(TRANSFORMER_test_np):
    anchors_TRANSFORMER.append(TRANSFORMER_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, test_length, time_convert(time_lapsed)), end="\r", flush=True)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


...

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


In [55]:
for index, elem in enumerate(anchors_TRANSFORMER):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: Grd_Prod_VoltPhse2_Avg <= 57227.10
Precision: 1.00
Coverage: 0.50
1
Anchor: Gen_RPM_Std <= 9433.10
Precision: 0.99
Coverage: 0.50
2
Anchor: Grd_Prod_VoltPhse2_Avg <= 57227.10
Precision: 1.00
Coverage: 0.51
3
Anchor: HVTrafo_Phase3_Temp_Avg <= 9698.00
Precision: 1.00
Coverage: 0.50
4
Anchor: HVTrafo_Phase3_Temp_Avg <= 9698.00
Precision: 1.00
Coverage: 0.50
5
Anchor: Grd_Prod_VoltPhse1_Avg <= 57483.00
Precision: 0.99
Coverage: 0.51
6
Anchor: HVTrafo_Phase3_Temp_Avg <= 11086.00
Precision: 0.98
Coverage: 0.75
7
Anchor: Grd_Prod_VoltPhse1_Avg <= 57562.10
Precision: 0.99
Coverage: 0.74
8
Anchor: Hyd_Oil_Temp_Avg <= 5095.00
Precision: 0.99
Coverage: 0.51
9
Anchor: Grd_Prod_VoltPhse2_Avg <= 57343.80
Precision: 0.99
Coverage: 0.75
10
Anchor: Grd_RtrInvPhase1_Temp_Avg > 5526.00
Precision: 0.97
Coverage: 0.50
11
Anchor: HVTrafo_Phase3_Temp_Avg <= 9698.00
Precision: 1.00
Coverage: 0.50
12
Anchor: Hyd_Oil_Temp_Avg <= 5863.00
Precision: 0.98
Coverage: 0.75
13
Anchor: Grd_Prod_VoltPhse2_Avg

In [ ]:
start_time = time.time()
for i, dp in enumerate(HYDRAULIC_GROUP_test_np):
    anchors_HYDRAULIC_GROUP.append(HYDRAULIC_explainer.explain(dp, threshold=0.95))
    current_time = time.time()
    time_lapsed = current_time - start_time
    print("{} von {} Schritten abgeschlossen. Zeit: {}".format(i, test_length, time_convert(time_lapsed)), end="\r", flush=True)

In [57]:
for index, elem in enumerate(anchors_HYDRAULIC_GROUP):
    print(index)
    print('Anchor: %s' % (' AND '.join(elem["data"]["anchor"])))
    print('Precision: %.2f' % elem["data"]["precision"])
    print('Coverage: %.2f' % elem["data"]["coverage"])

0
Anchor: Spin_Temp_Avg <= 3939.00 AND Grd_Prod_VoltPhse3_Avg > 57185.30
Precision: 0.96
Coverage: 0.36
1
Anchor: Spin_Temp_Avg <= 2706.00
Precision: 0.99
Coverage: 0.25
2
Anchor: Spin_Temp_Avg <= 2706.00
Precision: 0.97
Coverage: 0.25
3
Anchor: Spin_Temp_Avg <= 3939.00 AND Grd_Prod_VoltPhse1_Avg > 57483.00 AND Grd_Prod_CurPhse2_Avg <= 110283.00
Precision: 0.96
Coverage: 0.25
4
Anchor: Spin_Temp_Avg <= 3408.00
Precision: 0.96
Coverage: 0.50
5
Anchor: Grd_Prod_VoltPhse1_Avg <= 57417.10 AND Grd_Prod_CurPhse1_Avg > 41073.40 AND Amb_Temp_Avg > 2333.00
Precision: 0.95
Coverage: 0.06
6
Anchor: Grd_Prod_VoltPhse2_Avg > 57343.80
Precision: 0.96
Coverage: 0.25
7
Anchor: Grd_Prod_VoltPhse2_Avg > 57227.10 AND Avg_Windspeed2 > 4.01 AND Spin_Temp_Avg <= 3939.00 AND Grd_Prod_VoltPhse1_Avg <= 57562.10
Precision: 0.96
Coverage: 0.14
8
Anchor: Grd_Prod_VoltPhse2_Avg > 57343.80
Precision: 0.98
Coverage: 0.24
9
Anchor: Spin_Temp_Avg <= 3408.00
Precision: 0.98
Coverage: 0.49
10
Anchor: Spin_Temp_Avg <= 34